In [275]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from matplotlib import pyplot
from newspaper import Article

%matplotlib inline

from collections import defaultdict
from datetime import timedelta, date, datetime
import json
import numpy as np

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity="all"


In [276]:
data_file_names = ['grossveranstaltungen', 'sport-fussball',
              'unfaelle', 'demonstrationen',
              'immigration', 'straftaten-anschlaege'
                  ]

In [277]:
data = {}
for name in data_file_names:
    with open(name+".json") as data_file:
        topic_data = json.load(data_file)
        data[name] = topic_data

In [278]:
client = Elasticsearch(hosts='robin-in.space', timeout=30)

In [279]:
class ArticleQueryBuilder():
    def __init__(self, article):
        self.article = article

    def baseline(self):
        return Q("match", title=self.article['title']) & Q("match", body=self.article['title'])
    
    def simple_and_query(self):
        return Q("match", title=self.article['title']) & Q("match", body=self.article['body'])
        
        
    def simple_or_query(self):
        return Q("match", title=self.article['title']) | Q("match", body=self.article['body'])
    
    
    def date_filter(self, before=timedelta(weeks=2),
                            after=timedelta(weeks=8)):
        date = datetime.strptime(self.article['date'], '%Y-%m-%d')
        return Q("bool", filter=[Q('range', 
                            published={
                                'gte': date - before,
                                'lte': date + after
                            })])
    
    def location_query(self):
        return Q("bool", should=[
            Q("match", body=self.article['location']), 
            Q("match", title=self.article['location']),
            Q("match", officeName=self.article['location'])])

In [15]:
#Beispiel wie die Suche funktioniert
article= data['demonstrationen']['data'][0]['articles'][0]
ab= ArticleQueryBuilder(article)
query = ab.simple_and_query() & ab.location_query() & ab.date_filter()

In [40]:
s = Search(using=client)
res = s.query(query).execute()

In [149]:
#Permutiertation der Zahlen von 0 - 9 erstellen und ausgeben
array=np.random.permutation(10)
print(array)

[2 3 0 9 1 7 5 6 4 8]


In [269]:
#Für die Erstelleung des NDCG werden für jeden Nachrichtenartikel für zwei verschiedene Anfragen die obersten k Treffer in zufälliger Reihenfolge angezeigt, per Hand bewertet und anschließend das Verfahren per Hand bewertet
topic="immigration"
len(data[topic]['data'])
ereignisNummer=1
len(data[topic]['data'][ereignisNummer]['articles'])
artikelNummer=0
data[topic]['data'][ereignisNummer]['articles'][artikelNummer]['url']



5

1

'https://freie-presse.net/migranten-poebeln-kunden-an-und-erhalten-hausverbot/'

In [270]:
#alle relevanten PR zu dem aktuellen Artikel
for value in data[topic]['data'][ereignisNummer]['pr-ids']:
    print(value)

CuO3VmUB_NHj735h8IvE


In [274]:
#die ersten k Treffer der beiden Suchanfragen werden in zufälliger Reihenfolge angezeigt
article = data[topic]['data'][ereignisNummer]['articles'][artikelNummer]
ab= ArticleQueryBuilder(article)
queryBorder=ab.borderline()
queryImprove= ab.simple_and_query() & ab.location_query() & ab.date_filter()
s = Search(using=client)
res1=s.query(queryBorder).execute()
res2=s.query(queryImprove).execute()

k=3
permutation=np.random.permutation(2*k)
#print(permutation)
for i in range(2*k):
    j=int(permutation[i])
    if j>=k:
        print(res2[j-k].meta.id)
        print(res2[(j-k)].title)
        print(res2[(j-k)].body)
        print()
        print("____________________________________________________")
        print("----------------------------------------------------")
        print()
    
    else:
        print(res1[(j)].meta.id)
        print(res1[(j)].title)
        print(res1[(j)].body)
        print()
        print("____________________________________________________")
        print("----------------------------------------------------")
        print()

GET http://robin-in.space:9200/_search [status:N/A request:0.046s]
Traceback (most recent call last):
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/urllib3/connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/urllib3/util/connection.py", line 79, in create_connection
    raise err
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/urllib3/util/connection.py", line 69, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/elasticsearch/connection/http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
  File "/home/

GET http://robin-in.space:9200/_search [status:N/A request:0.103s]
Traceback (most recent call last):
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/urllib3/connection.py", line 171, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/urllib3/util/connection.py", line 79, in create_connection
    raise err
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/urllib3/util/connection.py", line 69, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/julius/police-pr-search/.venv/lib/python3.5/site-packages/elasticsearch/connection/http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
  File "/home/

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f812e9b95f8>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f812e9b95f8>: Failed to establish a new connection: [Errno 111] Connection refused)

In [268]:
#Ausgabe der besten k Treffer der beiden Suchanfragen in der richtigen Reihenfolge
print("Borderline IDs")
for i in range(k):
    print(res1[i].meta.id)
        
print()
print("----------------------------------------------------")
print()

print("Improved IDs")
for i in range(k):
    print(res2[i].meta.id)

Borderline IDs
MPPNVmUB_NHj735h7H1f
tOe8VmUB_NHj735h_8Ik
r-i9VmUB_NHj735hXA7s

----------------------------------------------------

Improved IDs
D_PNVmUB_NHj735h7JNi
8Oe8VmUB_NHj735h_8sl
C-i9VmUB_NHj735hXCTu


In [173]:
def construct_queries(article):
    builder = ArticleQueryBuilder(article)
    return {
        'and': builder.simple_and_query(),
        'or':builder.simple_or_query(),
        'and & date':builder.simple_and_query() & builder.date_filter(),
        'or & date':builder.simple_or_query() & builder.date_filter(),
        'and & loc':builder.simple_and_query() & builder.location_query(),
        'or & loc':builder.simple_or_query() & builder.location_query(),
        'and | loc':builder.simple_and_query() | builder.location_query(),
        'or | loc':builder.simple_or_query() | builder.location_query(),
        'and & loc & date':builder.simple_and_query() & builder.location_query() & builder.date_filter(),
        'or & loc & date':builder.simple_or_query() & builder.location_query() & builder.date_filter(),
        '(and | loc) & date':(builder.simple_and_query() | builder.location_query()) & builder.date_filter(),
        '(or | loc) & date':(builder.simple_or_query() | builder.location_query()) & builder.date_filter()
    }

In [56]:
def eval_query(query, pr_ids, client):
    res = Search(using=client).query(query).execute()
    print(res)
    return [(index, hit.meta.id) for index, hit in enumerate(res) if hit.meta.id in pr_ids]

In [23]:
def eval_event(event, client):
    results = defaultdict(dict)
    for article in event['articles']:
        queries = construct_queries(article)
        for name, query in queries.items(): 
            res = eval_query(query, event['pr-ids'], client)
            results[article['title']][name] = res
    return results

In [24]:
def eval_topic(topic, client):
    return [eval_event(event, client) for event in topic]

In [31]:
topic_results_2 = {topic_name: eval_topic(topic['data'], client) for 
                 topic_name, topic in data.items()}

In [282]:
topic_results_2.keys()

NameError: name 'topic_results_2' is not defined

In [280]:
def relevant_upto_k(query_result, k):
    relevant = 0
    for hit in query_result:
        rank, _ = hit
        if rank + 1 <= k:
            relevant = 1
    return relevant

In [284]:
def calc_mean_relevant_upto_k_by_topic(topic_result, k):
    query_result = []
    for event_result in topic_result:
        for title, query_dict in event_result.items():
            for query_type, results in query_dict.items():
                query_results.append(results)
                
    return sum(relevant_upto_k(result) for result in query_results) / len(query_results)
        
    

In [394]:
def precision_at_k(query_result, k):
    precision = 0
    for hit in query_result:
        rank, _ = hit
        if rank + 1 <= k:
            precision += 1
    return precision / k # We assume that always 
                         # at least k documents have been retrieved

In [395]:
def calc_mean_precision_at_k_by_topic(topic_result, k):
    query_results = []
    for event_result in topic_result:
        for title, query_dict in event_result.items():
            for query_type, results in query_dict.items():
                query_results.append(results)
    
    return sum(precision_at_k(result) for result in query_results) \
        / len(query_results)

In [281]:
precision_at_1_by_topic = {
    topic: calc_precision_at_k_by_topic(topic_results_2[topic], 1)
    for topic in data_file_names
}

NameError: name 'calc_precision_at_k_by_topic' is not defined

In [411]:
precision_at_1_by_topic

{'grossveranstaltungen': 0.8333333333333334,
 'sport-fussball': 0.71875,
 'unfaelle': 0.8095238095238095,
 'demonstrationen': 0.6666666666666666,
 'immigration': 0.4,
 'straftaten-anschlaege': 0.7878787878787878}

In [398]:
def split_topic_results_into_query_types(topic_results):
    query_type_results = defaultdict(list)
    for topic_name, topic_result in topic_results.items():
        for event_result in topic_result:
            for title, query_dict in event_result.items():
                for query_type, results in query_dict.items():
                    query_type_results[query_type].append(results)
    return query_type_results

In [434]:
len([el for el in split_topic_results_into_query_types(topic_results_2)['or'] if len(el) > 0 and el[0][0]==0])

35

In [434]:
len([el for el in split_topic_results_into_query_types(topic_results_2)['or'] if len(el) > 0 and el[0][0]==0])

35

In [428]:
split_topic_results_into_query_types(topic_results_2)['or']

[[(0, 'NfDKVmUB_NHj735hjcZN')],
 [(0, 'NfDKVmUB_NHj735hjcZN')],
 [(0, 'NfDKVmUB_NHj735hjcZN')],
 [(0, 'vui9VmUB_NHj735h56bE')],
 [(0, 'T_LMVmUB_NHj735h9bsl')],
 [(0, 'T_LMVmUB_NHj735h9bsl')],
 [(0, 'OuW5VmUB_NHj735h9z8O')],
 [(0, 'gfHLVmUB_NHj735hCSQU')],
 [(0, 'dfHLVmUB_NHj735hiJ5E')],
 [],
 [(0, '4e3FVmUB_NHj735hFq4M')],
 [],
 [(0, 'y-a7VmUB_NHj735htL_I')],
 [(1, '8fTOVmUB_NHj735h2zuh')],
 [(0, '1vLMVmUB_NHj735hR0dY')],
 [(0, 'puvCVmUB_NHj735hPtV-')],
 [(0, 'zem-VmUB_NHj735hkDdN')],
 [(0, 'GPTOVmUB_NHj735hmham')],
 [(0, 'kPDKVmUB_NHj735hjbxL')],
 [(0, 'v_LMVmUB_NHj735hwKmH')],
 [(0, 'yfDJVmUB_NHj735hqEGf'), (2, 'ZvDJVmUB_NHj735h-GUG')],
 [(0, 'HOe9VmUB_NHj735hLOoi'), (5, 'z-W6VmUB_NHj735hUKfk')],
 [],
 [],
 [(0, 'tei9VmUB_NHj735hjjjS')],
 [(0, 'ceO4VmUB_NHj735hRc0c')],
 [(0, 'ceO4VmUB_NHj735hRc0c')],
 [(0, 'ceO4VmUB_NHj735hRc0c')],
 [(0, 'zvDJVmUB_NHj735h-HEI')],
 [(0, 'WO3DVmUB_NHj735h4Aq9')],
 [(6, 'kue8VmUB_NHj735h_8Ik')],
 [(0, 'CuO3VmUB_NHj735h8IvE')],
 [(1, 'zu7GVmUB_NHj735hoHP

In [441]:
def calc_precision_at_k_by_query_type(topic_results, k):
    query_type_results = split_topic_results_into_query_types(topic_results)
    precisions = {}
    for query_type, results_by_type in query_type_results.items():
        sum_precision_at_k = 0
        for results in results_by_type:
            sum_precision_at_k += precision_at_k(results, k)
        precisions[query_type] = sum_precision_at_k / len(results_by_type)
    return precisions

In [451]:
calc_precision_at_k_by_query_type(topic_results_2, 3)

{'and': 0.2753623188405797,
 'or': 0.31884057971014496,
 'and & date': 0.2971014492753623,
 'or & date': 0.33333333333333337,
 'and & loc': 0.3043478260869565,
 'or & loc': 0.34782608695652173,
 'and | loc': 0.29710144927536225,
 'or | loc': 0.34057971014492755,
 'and & loc & date': 0.32608695652173914,
 'or & loc & date': 0.3695652173913043,
 '(and | loc) & date': 0.31884057971014496,
 '(or | loc) & date': 0.36231884057971014}

In [ ]:
def calc_mean_mrr_by_query(topic_results):
    pass

In [362]:
topic_results

{'grossveranstaltungen': [defaultdict(dict,
              {'Rhein in Flammen: Polizei ermittelt wegen sexueller Übergriffe': {'and': [(0,
                  'NfDKVmUB_NHj735hjcZN')],
                'or': [(0, 'NfDKVmUB_NHj735hjcZN')],
                'and & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (2, 'jPDKVmUB_NHj735hjb5L')],
                'or & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (3, 'jPDKVmUB_NHj735hjb5L')],
                'and & loc': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (7, 'jPDKVmUB_NHj735hjb5L')],
                'or & loc': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (8, 'jPDKVmUB_NHj735hjb5L')],
                'and | loc': [(0, 'NfDKVmUB_NHj735hjcZN')],
                'or | loc': [(0, 'NfDKVmUB_NHj735hjcZN')],
                'and & loc & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (1, 'jPDKVmUB_NHj735hjb5L')],
                'or & loc & date': [(0, 'NfDKVmUB_NHj735hjcZN'),
                 (1, 'jPDKVmUB_NHj735hjb5L')

In [464]:
art = Article('http://www.wn.de/Muenster/3246264-Fahrzeug-faehrt-in-Menschenmenge-Tote-und-Verletzte-nach-Anschlag'
             ,language='de')

In [471]:
art.download()
art.parse()
art.nlp()

In [472]:
art.title

'Fahrzeug fährt in Menschenmenge Tote und Verletzte nach Anschlag'

In [475]:
art.summary

'Tote und Verletzte nach Anschlag am Kiepenkerl 1/53 Kleinlaster fährt in Menschenmenge Foto: privatKleinlaster fährt in Menschenmenge Foto: Oliver WernerMenschen stehen vor den Gaststätten Kiepenkerl, kurz nachdem ein Fahrzeug in das Straßencafe gefahren war.\nFoto: Ina Fassbender/dpaFoto: David Young/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerFoto: Bernd Thissen/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerAm Abend nach der Bluttat in Münster versammelten sich Menschen am Aasee und entzündeten Kerzen.\nFoto: Friso Gentsch/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerFoto: Bernd Thissen/dpaKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: Oliver WernerKleinlaster fährt in Menschenmenge Foto: